In [109]:
from os import listdir
from os.path import isfile, join
import cv2
import random
import numpy as  np
import math

def generateExpandedImage(image):
    # This function generates a white background image that is 104% larger (four times of sin(15))than the original image
    # In this program max rotation degree is +-15 degrees. sin(15 degree)<0.258
    (h, w) = image.shape[:2]
    (h_new, w_new) = (math.ceil(h * 2.1), math.ceil(w * 2.1))
    # print ("h = {}, w = {}, h_new = {}, w_new = {}".format(h,w, h_new, w_new))
    bg_img = np.zeros(shape=[h_new, w_new], dtype=np.uint8)
    bg_img = cv2.bitwise_not(bg_img)
    top_left_y = int((h_new - h)/2)
    top_left_x = int((w_new - w)/2)
    # print ("top_left_y = {}, top_left_x = {}".format(top_left_y,top_left_x))
    bg_img[top_left_y:h+top_left_y, top_left_x:w+top_left_x]=image
    return (bg_img, h, w)
    
def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def cutbackImage(image, h, w):
    (h_r,w_r) = image.shape[:2]
    h_c, w_c = math.ceil(h * 1.26), math.ceil(w * 1.26)
    # 1.26 comes from sin(15) = 02586
    # print ("h = {}, w = {}, h_c = {}, w_c = {}".format(h,w, h_c, w_c))
    # specify the area of top lef to lower right
    # math.ceil rounds up number into integer
    image_cut = image[math.ceil((h_r - h_c)/2):h_c+math.ceil((h_r - h_c)/2), math.ceil((w_r - w_c)/2):w_c+math.ceil((w_r - w_c)/2)]
    return image_cut, h_c, w_c

def resizeImage(image, h, w):
    h_resize = random.randint(40, 75) # target image size's height is between 40 and 75
    w_resize = math.ceil(w * h_resize / h)  # maintain the aspect ratio
    if w_resize > 130:
        w_resize = 130
        h_resize = math.ceil(h * 130/w)
    dim = (w_resize, h_resize)
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return result

def top_left (max_v, v):  # this function calculate top_left coordiates and works for both x and y
    rand = np.random.randn()  # generate random number with average 0 standard diviation 1
    while abs(rand) > 3:   # eliminate rare cases that exceed 3 sigma
        rand = np.random.randn()
    v_range = 2 * (math.floor(max_v/2)-math.ceil(v/2))
    if v_range < 0:  # manage rounding error
        v_range = 0
    # since random number generate between -3 and +3, the following equation divide the possible range by 6
    top_left = math.floor(max_v/2) - math.floor(v_range/6*rand)- math.floor(v/2)  
    return top_left


def placeImage (image, max_h, max_w): # place the character to max size randomely
    bg_img = np.zeros(shape=[max_h, max_w], dtype=np.uint8)
    bg_img = cv2.bitwise_not(bg_img)
    (h,w) = image.shape[:2]
    tl_w = top_left(max_w, w)
    tl_h = top_left(max_h, h)
    # print ("h = {}, w = {}, tl_h ={}, tl_w={}".format(h,w,tl_h,tl_w))
    bg_img[tl_h:h+tl_h, tl_w:w+tl_w] = image
    return bg_img
    
def resizeFinal(image, h, w):
    dim = (w, h)
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    result = cv2.bitwise_not(result)
    return result

rpath = "F:/Google Drive/User_Backup/base_font"
wpath = "F:/Google Drive/User_Backup/training"
h_max = w_max = 0
onlyfolders = [f for f in listdir(rpath) if f[-4:] != ".ini"]
for folder in onlyfolders:
    npath = rpath +"/"+ folder
    print(npath)
    onlyfiles = [f for f in listdir(npath) if isfile(join(npath, f)) and f[-4:] == ".png"]
    for i in range(5000):
        path = npath +"/"+ random.choice(onlyfiles)
        img = cv2.imread(path, 0)
        angle = random.randint(-15, 15) # range of rotation -15 to 15 degrees
        bg_img, h, w = generateExpandedImage(img) # add extra white blank around the original image
        result = rotateImage(bg_img, angle) # rotate image
        result, h, w = cutbackImage(result, h, w)  # cut unnecessary area and extract the rotated character
        result = resizeImage(result, h, w) # randomly resize the image
        max_h, max_w = 75, 132    # max size to map numbers
        result = placeImage(result, max_h, max_w) # put the iamge into the large white background
        train_h, train_w = 25, 44 # AI training data size
        result = resizeFinal(result, train_h, train_w) # shrink the final image for AI model training
        opath = wpath + "/" + folder + "/" + str(i) + ".png"
        # cv2.imshow('image', result)
        # cv2.waitKey()
        # cv2.destroyAllWindows()
        # print(opath)
        cv2.imwrite(opath,result)


F:/Google Drive/User_Backup/base_font/0
F:/Google Drive/User_Backup/base_font/1
F:/Google Drive/User_Backup/base_font/2
F:/Google Drive/User_Backup/base_font/3
F:/Google Drive/User_Backup/base_font/4
F:/Google Drive/User_Backup/base_font/5
F:/Google Drive/User_Backup/base_font/6
F:/Google Drive/User_Backup/base_font/7
F:/Google Drive/User_Backup/base_font/8
F:/Google Drive/User_Backup/base_font/9
